# Seleccionando palabras

In [1]:
import string
import numpy as np
import reuters_reader
from sklearn.cluster import MiniBatchKMeans
from collections import Counter
from sklearn.externals import joblib
from gensim.models.keyedvectors import KeyedVectors
from sklearn import linear_model

np.set_printoptions(precision=3)

### RCV1 Dataset

Use the function ```reuters_reader.reader(path)``` to retrieve the available documents from the rcv1 dataset stored in `path`. This function generates a document which is a dictionary with the data from an specific rcv1 entry. There are a total of 804420 available documents, although some may have no topic.

Some useful keys in this dicitionary are:
 - "title"
 - "text" which is the body of document
 - "bip:topics:1.0" is a list of topics

In [2]:
n_docs = 50000
docs = []
reader = reuters_reader.reader('rcv1')
#total_documents = 0
#for doc in reader:
#    total_documents += 1
#print('Total documents: {}'.format(total_documents))
print('Using: {}'.format(n_docs))
for i in range(n_docs):
    doc = next(reader)
    docs.append(doc)
    if i < 10:
        print(doc['title'])
        print(doc['bip:topics:1.0'])
        print(doc['text'][:100], doc['text'][-100:])

Using: 50000
None
['C12', 'CCAT', 'GCAT', 'GCRIM']
South African Airways (SAA) SAA.CN has accused domestic rival Comair, whose new franchise agreement   had obtained much more detailed information on passengers.

- Johannesburg newsroom +27 11 482 1003
None
['C11', 'CCAT']
German retail group Karstadt AG plans to move to a holding structure as soon as it completes its int .2 million marks in 1995 from a depressed 41.9 million in 1994.

--Frankfurt Newsroom, +49 69 756525
None
['E12', 'ECAT', 'GCAT', 'GPOL']
An Argentine bishop on Saturday attacked government plans to reform the labour market and said what  piness and creativity, that our pensioners lead a languid existence, nearly like a prolonged agony."
None
['GCAT', 'GVIO']
A Jewish settler fired shots at Arab houses in the West Bank town of Hebron on Sunday but caused no  

Settlers have said a partial Israeli army pullout from the town would put their lives in jeopardy.
None
['E12', 'E21', 'E211', 'ECAT', 'GCAT', 'GPOL']
Prime Min

### Word2vec model

We are going to load a well known word2vec model from __[Google](https://code.google.com/archive/p/word2vec/)__ which is stored in the binary file `GoogleNews-vectors-negative300.bin`.

With the model we define a numpy matrix `X`, which contains as much vectors as words are in the word2vec vocabulary. We need this dataset to train the kmeans algorithm.

Load the word2vec model

In [3]:
w2v_name = 'GoogleNews-vectors-negative300.bin'
w2v = KeyedVectors.load_word2vec_format(w2v_name, binary=True)

### Get our vocabulary

Get all the the vectors from the word2vec for our vocabulary. Our vocabulary can include all the words used in the word2vec model or be limited to the words in our dataset.

We can change the beahivour with the flag ```dataset_vocabulary```. ```False``` will use all the words from the word2vec model and ```True``` will limit them to just the words that are in our dataset and in the model at the same time.

There is a ```count_threshold``` to remove those words appearing very few times because they are errors.

After this cell, ```X``` is a matrix including all the vectors we are going to use. We also store the words for each document in ```doc["counter"]```. 

In [4]:
dataset_vocabulary = True
count_threshold = 5

if dataset_vocabulary:
    vocab = Counter()
    for doc in docs:
        doc["counter"] = Counter()
        words = doc["text"].split()
        words = [word.strip(string.punctuation) for word in words]
        for word in words:
            if word in w2v:
                doc["counter"][word] += 1
                vocab[word] += 1
    vocab = {word: count for word, count in vocab.items() if count > count_threshold}
else:
    vocab = w2v.index2word

X = np.zeros((len(vocab), w2v.vector_size), dtype=np.float32)
for index, word in enumerate(vocab):
    X[index, :] += w2v[word]
    
print("Vocabulary length: {}".format(X.shape[0]))
print("Vector length: {}".format(X.shape[1]))

Vocabulary length: 40726
Vector length: 300


Let's take a look at the most common and uncommon words in our dataset

In [5]:
print(Counter(vocab).most_common()[:10])
print(Counter(vocab).most_common()[-10:])

[('the', 518328), ('in', 225218), ('said', 157016), ('on', 128037), ('for', 100210), ('The', 94018), ('at', 76465), ('was', 71803), ('that', 67252), ('is', 66545)]
[('Perignon', 6), ('AGCO', 6), ('CONTEXT', 6), ('MFRI', 6), ('Novoste', 6), ('Kaminski', 6), ('OIS', 6), ('Bafokeng', 6), ('Stankevich', 6), ('MSG', 6)]


### Get the labels ###
Select a topic we want to classify using the variable ```topic``` (there is a list of the topics __[here](https://gist.github.com/gavinmh/6253739)__). Then build the list of labels using a ```1``` for those documents with that topic and 0 otherwise 

In [6]:
topic = 'ECAT'

labels = np.zeros((n_docs), dtype=np.int16)
labels = [1 if topic in doc['bip:topics:1.0'] else 0 for doc in docs]

print('{} docs with topic {} (from {})'.format(np.sum(labels), topic, n_docs))

7452 docs with topic ECAT (from 50000)


### Initial Cluster

Train the first Kmeans cluster

In [7]:
n_clusters = 50
kmeans_name = 'kmeans' + str(n_clusters) + '.pkl'

In [8]:
classifier = MiniBatchKMeans(n_clusters=n_clusters, random_state=0, compute_labels=True)
classifier.fit(X)
joblib.dump(classifier, kmeans_name)

['kmeans50.pkl']

In [9]:
# def print_clusters(classifier, w2v, topn=3):
# #     w2v.init_sims(replace=True)
#     for (i, center) in enumerate(classifier.cluster_centers_):
#         #print(center)
#         similar = w2v.wv.most_similar_cosmul(positive=[center], topn=topn)
#         similar = [t[0] for t in similar]
#         print("Cluster %d: %s" % (i + 1, ', '.join(similar)))
        
#print_clusters(classifier, w2v)    


### Build a bow model 

Using the clusters from the kmeans classifier build a bow for each document. This bag of words can be normalized usign the frequency of each word with ```useFrequency=True```. 

How? 
* For each document
    * For each word
        * Obtain the w2v vector for that word
        * Obtain the cluster for that vector
        * Add 1 to that cluster in the document bow

To improve the performance the cluster for each word is saved in a dictionary. This way for each word we first check that dictionary instead of the model and then the classifier.


In [10]:
def clusters_bow(docs, classifier, w2v, useFrequency=True, printing=False):
    n_docs = len(docs)
    n_clusters = classifier.n_clusters
    bo_clusters = np.zeros((n_docs, n_clusters))

    hashed_clusters = {}
    for (i, doc) in enumerate(docs):
        for word, count in doc["counter"].items():
            if word in hashed_clusters:
                cluster = hashed_clusters[word]
            else:
                cluster = classifier.predict([w2v[word]])[0]
                hashed_clusters[word] = cluster
            bo_clusters[i][cluster] += count
            
    if useFrequency:
        sums = np.sum(bo_clusters, axis=1)
        boc = bo_clusters / sums.reshape((sums.shape[0], 1))
    else:
        idx = bo_clusters != 0
        # Add a small amount to the denominator to avoid zero division
        boc = np.round(bo_clusters / (bo_clusters + 0.001))
        
    if printing:
        for i in range(10):
            print('Document %d: %s' % (i + 1, np.array2string(boc[i, :])))
        
    return boc

# bows = clusters_bow(docs, classifier, w2v)

Use these bows to classify

### Keep, split or discard clusters

Once we have our bags of words we can study how good are our clusters. To do so we use a Lasso function feeding it with the bows. 

From this function we obtain different coefficients that we will use to decide how good is a particular cluster. Only clusters with a coefficient higher than ```deactivate_value``` are kept, although between these some must be splitted. Splitted clusters are those with a coefficient lower than ```split_threshold``` times the maximum coefficient. 

In [11]:
def study_clusters(bows, labels, deactivate_value=0.1, split_threshold=0.6, printing=False):  
    clf = linear_model.LassoCV()
    clf.fit(bows, labels)
    
    max_w = np.max(np.abs(clf.coef_))
    split_value = deactivate_value * max_w

    if printing:
        print('Lasso coefficients: %s' % (np.array2string(clf.coef_, suppress_small=True)))
        print('Split value (using threshold %.2f): %.2f' % (split_threshold, split_value))
    
    # deactivate = [x <= deactivate_value for x in np.abs(clf.coef_)]
    split = [x > deactivate_value and x <= split_value for x in np.abs(clf.coef_)]
    keep = [x > split_value for x in np.abs(clf.coef_)]
    
    return keep, split

# keep, split = study_clusters(bows, labels)

# print('Keeping %d clusters' % (len([x for x in keep if x == True])))
# print('Spliting %d clusters' % (len([x for x in split if x == True])))

# new_count = len([x for x in keep if x == True]) + 2 * len([x for x in split if x == True])
# print('Using %d clusters in the next iteration' % (new_count))

Once we know which clusters to keep and which to split, we can build the new ones. 

To split one cluster we need to select all the points belonging to that cluster and classify then using 2 neighbourds. 

In [12]:
def update_centers(classifier, X, keep, split):
    n_centers = len([x for x in keep if x == True]) + 2 * len([x for x in split if x == True])
    updated_centers = np.empty((n_centers, classifier.cluster_centers_.shape[1]))
    new_idx = 0
    for i in range(len(keep)):
        if keep[i]:
            updated_centers[new_idx, :] = classifier.cluster_centers_[i, :]
            new_idx += 1
        if split[i]:
            # create classifier with this data
            newX = X[np.where(classifier.labels_ == i)[0], :]
            small_class = MiniBatchKMeans(n_clusters=2, random_state=0, compute_labels=True)
            small_class.fit(newX)
            updated_centers[new_idx, :] = small_class.cluster_centers_[0, :]
            updated_centers[new_idx + 1, :] = small_class.cluster_centers_[1, :]
            new_idx += 2
            
    return updated_centers
        
# new_centers = update_centers(classifier, X, keep, split)
    
# print(classifier.cluster_centers_)
# print(new_centers)
# print(classifier.counts_)
# print(classifier.labels_)
# print(classifier.labels_.shape)

### Build a new classifier

We have to use the new clusters to classify the data in the following steps. One possible to solution to use these new clusters is to update the classifier centers and the relevant attributes. Then we can use the classifier's function ```predict``` as before. Moreover, to keep using this classifier to build the next cluster we also need to update the ```labels_``` and ```counts_``` parameters. 

In [13]:
def update_classifier(classifier, new_centers, x):
    classifier.cluster_centers_ = new_centers
    classifier.n_clusters = len(new_centers)
    classifier.labels_, _ = classifier._labels_inertia_minibatch(x)
    classifier.counts_ = np.zeros(classifier.n_clusters, dtype=np.int32)
    for i in range(classifier.n_clusters):
        classifier.counts_[i] = np.sum(classifier.labels_ == i)
    
    return classifier

# classifier = update_classifier(classifier, new_centers, X)

In [14]:
epochs = 10;
original_classifier = classifier;

for i in range(epochs):
    print('Epoch %d of %d' % (i + 1, epochs))
    print('Number of clusters: %d' % (classifier.n_clusters))
    bows = clusters_bow(docs, classifier, w2v, printing=True)
    keep, split = study_clusters(bows, labels, printing=True)

    print('Keeping %d clusters' % (len([x for x in keep if x == True])))
    print('Spliting %d clusters' % (len([x for x in split if x == True])))
    
    next_centers = update_centers(classifier, X, keep, split)
    
    classifier = update_classifier(classifier, next_centers, X)
#     print_clusters(classifier, w2v)  
    

Epoch 1 of 10
Number of clusters: 50
Document 1: [ 0.     0.     0.     0.007  0.     0.289  0.     0.018  0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.054  0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.004
  0.     0.     0.     0.     0.     0.629  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.   ]
Document 2: [ 0.     0.     0.     0.013  0.     0.16   0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.103  0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.019
  0.     0.     0.     0.     0.     0.705  0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.   ]
Document 3: [ 0.     0.     0.     0.006  0.     0.246  0.     0.012  0.     0.     0.006
  0.     0.     0.     0.     0.     0.     0.     0.03   0.     0.006  0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.024
  

/home/sasi/.anaconda3/envs/words/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso coefficients: [  0.     -1.751  -3.353  -0.252  -0.      0.443  -0.     -1.465  -1.043
  -0.      1.443   0.      0.761   0.      0.      0.152  -0.      0.     -0.3
  -0.      1.96   -0.      3.036  -0.      1.719  -0.     -0.      0.
  20.94   -0.     -0.     -0.      0.376   0.     -0.     -2.509  -0.
   0.729  -0.      0.684  -2.3     0.      0.      0.     -0.     -0.     -0.
  -0.     -0.     -1.782]
Split value (using threshold 0.60): 2.09
Keeping 5 clusters
Spliting 15 clusters
Epoch 2 of 10
Number of clusters: 35
Document 1: [ 0.     0.     0.     0.007  0.     0.657  0.175  0.025  0.011  0.     0.
  0.     0.     0.     0.     0.     0.     0.014  0.05   0.     0.     0.
  0.     0.007  0.     0.018  0.036  0.     0.     0.     0.     0.     0.
  0.     0.   ]
Document 2: [ 0.     0.     0.     0.006  0.032  0.731  0.09   0.019  0.     0.     0.
  0.006  0.     0.     0.     0.     0.     0.083  0.     0.     0.     0.
  0.     0.     0.     0.     0.032  0.     0.     

/home/sasi/.anaconda3/envs/words/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso coefficients: [ -2.522  -1.488  -0.     -0.052  -1.096  -0.088   1.118   0.508  -3.058
  -0.969   0.979   1.895   1.234   0.37    0.896  -2.099  -0.586   0.023
  -0.26   -0.463   1.626  -0.      1.343   0.582  15.281   0.238   0.49
  -2.379   7.161   0.      0.215  -0.     -2.312  -3.338  -1.442]
Split value (using threshold 0.60): 1.53
Keeping 10 clusters
Spliting 18 clusters
Epoch 3 of 10
Number of clusters: 46
Document 1: [ 0.     0.     0.     0.     0.     0.186  0.55   0.     0.057  0.043  0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.     0.     0.036  0.     0.     0.     0.     0.004  0.     0.007
  0.004  0.     0.054  0.004  0.004  0.054  0.     0.     0.     0.     0.
  0.     0.     0.   ]
Document 2: [ 0.     0.     0.019  0.     0.09   0.244  0.455  0.     0.013  0.013  0.
  0.     0.     0.     0.006  0.     0.     0.006  0.     0.     0.     0.
  0.     0.     0.026  0.     0.     0.     0.     0.006  0.     0.     0.013
  0.  

...

/home/sasi/.anaconda3/envs/words/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Lasso coefficients: [ -0.934  -1.104   1.048   0.413   1.27    0.038  -0.086   0.191  -0.055
   1.557  -1.615   0.28   -0.574  -0.886  -0.442  -0.017  -0.916   1.241
  -0.481  -0.619   0.486   1.664   0.335  -1.605   0.5     1.979   1.256
  -0.     -0.597   0.36    0.578   1.461   0.751   2.834   0.722  -0.818
  -0.346  10.633  -1.638   0.618  -0.178   0.244  -0.651  -0.305  -0.032
   0.94    2.031   1.178   3.222  -0.     -0.187  -1.58   -0.305  -0.46
  -2.16    0.681   0.793  -0.99    7.559  -1.785  -2.967  -0.919]
Split value (using threshold 0.60): 1.06
Keeping 21 clusters
Spliting 34 clusters
Epoch 7 of 10
Number of clusters: 89
Document 1: [ 0.     0.     0.     0.     0.     0.     0.     0.089  0.025  0.493  0.
  0.004  0.     0.     0.     0.     0.     0.     0.     0.     0.018
  0.007  0.061  0.039  0.054  0.025  0.036  0.     0.     0.     0.     0.004
  0.     0.     0.     0.     0.007  0.     0.     0.     0.     0.     0.004
  0.     0.     0.     0.     0.011  0.029  

Lasso coefficients: [ -0.419   2.029   1.468  -0.124  -0.443  -0.216  -0.051   0.013   0.905
   1.028  -1.425   0.426   0.787   0.5    -0.945   5.002  -0.44   -2.861
  -3.119  -0.236  -0.706  -0.891  -0.271  -0.121   0.283  -0.107  -0.828
  -0.     -0.561  -0.5    -1.132  -0.112  -1.865   0.365   3.173   0.667
  -0.862  -0.54   -0.857   0.402   0.043  -0.046   1.612  -0.597  -0.      0.8
   1.724   0.784  -1.464   0.911  -0.     -0.     -0.      1.788   2.916
   0.493  -0.     -0.     -0.     -0.009   0.021   0.315   3.106  -0.761
  -0.603  14.151   0.478   0.734   1.622   1.19    1.722   3.722  -0.686
   2.552  -0.777   0.786  -0.202  -0.198   0.039   0.25   -3.679   0.392
   1.796  -0.319   9.515  -1.91    0.908   0.633  -0.026  -0.225  -0.543
   0.812   1.072  -0.728  -0.241  -0.769   0.162   0.787  -1.286  -1.265
   0.668   0.898  -0.969   0.488   2.358   1.336   5.41   -0.515   0.192
  -1.562   0.565   1.539  -1.312  -0.456   0.035  -0.212  -1.96    1.783
  -1.283  -0.23    1.8   

Lasso coefficients: [ -0.     -0.181  -0.756   2.227  -0.89    0.171  -0.451  -0.102  -0.036
  -1.385   0.499   0.502   0.948   3.014  -0.894  -0.278  -0.627   0.22
   0.464  -0.245   0.579  -0.482   1.096   1.144  -0.251  -0.706  -0.844
  -1.954  -0.286   0.392  -0.     -0.131  -0.743  -0.031  -0.     -0.149
  -0.     -0.4     0.115   0.085  -0.215   0.     -0.198  -0.     -1.107
  -0.      0.106  -0.959  -1.116  -0.494   0.052   0.221  -0.652   0.724
  -0.69    2.916   0.827  -1.501  -0.407  -0.492  -0.218  -0.549   0.036
  -0.626   2.276  -0.122   0.     -0.275  -0.329  -0.      1.36    1.3     0.
  -0.765  -3.666   0.016   0.826   1.064   2.089  -2.786   0.149  -0.054
   0.399   3.248  -1.44   -0.791  -1.353  -0.717  13.819   0.166   0.679
  -1.069   1.404   0.5     1.981  -0.      2.145   0.     -0.411   0.024
   2.246  -1.233  -0.262  -0.      0.952   0.169  -0.484  -0.068   0.31
   0.352  -0.136  -2.644  -0.      1.212   1.278  -0.203   6.347  10.591
  -1.762   7.309   0.334  -1

Lasso coefficients: [  0.893   0.193  -0.227   2.305   2.998  -1.266  -0.381   0.323  -0.
  -0.631  -0.271  -0.152  -0.466  -0.      0.624   0.415   0.231   0.6
   1.935  -1.403   3.006  -0.645  -0.513  -1.478   2.653  -0.892  -0.737
  -1.139   0.833   0.538   0.365   2.71   -0.638   0.445   0.301  -0.
  -0.781   0.      0.     -0.      0.115  -0.595   0.509  -0.     -0.
  -0.726  -0.     -3.348   0.     -1.142   2.122   0.673  -0.61   -0.565
  -0.     -1.043  -0.291  -0.     -0.     -0.014  -0.045  -0.573  -0.756
   1.319  -0.     -0.04   -0.746  -0.338   0.     -0.     -0.61   -0.431
  -0.674  -1.38   -0.707  -0.843   0.167   0.176  -0.627  -0.182  -0.062
   3.057   0.09   -1.963   2.743   0.494   1.025  -1.95    0.061  -0.493
   0.     -0.511   0.15   -0.654  -0.647  -0.241   0.264  -0.463   2.269
  -2.538  -0.002   0.686  -0.288  -0.319  -0.      1.714  -0.583  -1.224
   0.894  -1.311   0.678  -3.442  -0.      0.53    2.635   3.467   1.914
  -3.794   0.517  -0.741   0.215   0.383  